# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 15 2022 2:47PM,253215,16,8906617,Sartorius Bioprocess Solutions,Released
1,Dec 15 2022 2:43PM,253214,10,SONSB0002053,"Nextsource Biotechnology, LLC",Released
2,Dec 15 2022 2:43PM,253214,10,SONSB0002054,"Nextsource Biotechnology, LLC",Released
3,Dec 15 2022 2:31PM,253212,10,Enova-11108,Emerginnova,Released
4,Dec 15 2022 2:24PM,253211,10,Yusen-8906636,Yusen – 3D Matrix,Released
5,Dec 15 2022 2:24PM,253211,10,Yusen-8906640,Yusen – 3D Matrix,Released
6,Dec 15 2022 2:24PM,253211,10,Yusen-8906642,Yusen – 3D Matrix,Released
7,Dec 15 2022 2:24PM,253211,10,Yusen-8906643,Yusen – 3D Matrix,Released
8,Dec 15 2022 2:24PM,253211,10,Yusen-8906644,Yusen – 3D Matrix,Released
9,Dec 15 2022 2:22PM,253210,10,0086251100,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,253210,Released,7
27,253211,Released,5
28,253212,Released,1
29,253214,Released,2
30,253215,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253210,NaN,NaN,7.0
253211,NaN,NaN,5.0
253212,NaN,NaN,1.0
253214,NaN,NaN,2.0
253215,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253139,0.0,0.0,1.0
253149,5.0,23.0,3.0
253179,0.0,0.0,1.0
253181,0.0,0.0,1.0
253184,0.0,10.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253139,0,0,1
253149,5,23,3
253179,0,0,1
253181,0,0,1
253184,0,10,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253139,0,0,1
1,253149,5,23,3
2,253179,0,0,1
3,253181,0,0,1
4,253184,0,10,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253139,,,1
1,253149,5,23,3
2,253179,,,1
3,253181,,,1
4,253184,,10,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 15 2022 2:47PM,253215,16,Sartorius Bioprocess Solutions
1,Dec 15 2022 2:43PM,253214,10,"Nextsource Biotechnology, LLC"
3,Dec 15 2022 2:31PM,253212,10,Emerginnova
4,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix
9,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation
16,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation
18,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC"
19,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc"
20,Dec 15 2022 1:22PM,253205,10,ISDIN Corporation
36,Dec 15 2022 1:15PM,253204,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 15 2022 2:47PM,253215,16,Sartorius Bioprocess Solutions,,,1
1,Dec 15 2022 2:43PM,253214,10,"Nextsource Biotechnology, LLC",,,2
2,Dec 15 2022 2:31PM,253212,10,Emerginnova,,,1
3,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,,,5
4,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,,,7
5,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,,,2
6,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",,,1
7,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",,,1
8,Dec 15 2022 1:22PM,253205,10,ISDIN Corporation,,,16
9,Dec 15 2022 1:15PM,253204,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 2:47PM,253215,16,Sartorius Bioprocess Solutions,1,,
1,Dec 15 2022 2:43PM,253214,10,"Nextsource Biotechnology, LLC",2,,
2,Dec 15 2022 2:31PM,253212,10,Emerginnova,1,,
3,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5,,
4,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7,,
5,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,2,,
6,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",1,,
7,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",1,,
8,Dec 15 2022 1:22PM,253205,10,ISDIN Corporation,16,,
9,Dec 15 2022 1:15PM,253204,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 2:47PM,253215,16,Sartorius Bioprocess Solutions,1,,
1,Dec 15 2022 2:43PM,253214,10,"Nextsource Biotechnology, LLC",2,,
2,Dec 15 2022 2:31PM,253212,10,Emerginnova,1,,
3,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5,,
4,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 2:47PM,253215,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Dec 15 2022 2:43PM,253214,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
2,Dec 15 2022 2:31PM,253212,10,Emerginnova,1.0,NaN,NaN
3,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5.0,NaN,NaN
4,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 2:47PM,253215,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Dec 15 2022 2:43PM,253214,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
2,Dec 15 2022 2:31PM,253212,10,Emerginnova,1.0,0.0,0.0
3,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5.0,0.0,0.0
4,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3798010,66.0,12.0,0.0
12,759607,1.0,2.0,0.0
16,759575,3.0,0.0,0.0
19,759543,5.0,23.0,5.0
20,253139,1.0,0.0,0.0
21,506391,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3798010,66.0,12.0,0.0
1,12,759607,1.0,2.0,0.0
2,16,759575,3.0,0.0,0.0
3,19,759543,5.0,23.0,5.0
4,20,253139,1.0,0.0,0.0
5,21,506391,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,66.0,12.0,0.0
1,12,1.0,2.0,0.0
2,16,3.0,0.0,0.0
3,19,5.0,23.0,5.0
4,20,1.0,0.0,0.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,66.0
1,12,Released,1.0
2,16,Released,3.0
3,19,Released,5.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,5.0,0.0,0.0
Executing,12.0,2.0,0.0,23.0,0.0,0.0
Released,66.0,1.0,3.0,5.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,12.0,2.0,0.0,23.0,0.0,0.0
2,Released,66.0,1.0,3.0,5.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,12.0,2.0,0.0,23.0,0.0,0.0
2,Released,66.0,1.0,3.0,5.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()